<h1 style="font-family:verdana;"> <center> RetinaNet 🎖</center> </h1>

<h3><center style="color:#159364; font-family:cursive;">💬 KICK-START this competition with RetinaNet</center></h3>

<img src="https://lh3.googleusercontent.com/1TaNTTbCmw4E94xU7XKdTQ-CR6t7lnqA_YLgXosi2S3_1eDzehkcjv3qnzkxjKmUOzxKzxqHg45vql5G8wChVh4B6W91GjQTTnV3PZybyMAZSch2n6ckY0P9sqKUbrxOwrCnSq-J">

<br>

📌  Special Thanks to
* https://github.com/fizyr/keras-retinanet
* https://www.kaggle.com/awsaf49 for the (.png) Dataset

<hr>

In [ ]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import cv2

from sklearn.model_selection import StratifiedKFold
import warnings

<h2 style="font-family:verdana;"> <center>📂 Config File</center> </h2>

<hr>

In [ ]:
class Cfg(object):
  
  def __init__(self):
    super(Cfg, self).__init__()
    self.dim = 512
    self.batch= 8
    self.steps= 500
    self.epochs= 10
    self.train_csv= '../input/vinbigdata-{}-image-dataset/vinbigdata/train.csv'.format(self.dim)
    self.test_csv= '../input/vinbigdata-{}-image-dataset/vinbigdata/test.csv'.format(self.dim)
    self.img_dir= '../input/vinbigdata-{}-image-dataset/vinbigdata/train/'.format(self.dim)
    
    self.git= 'https://github.com/fizyr/keras-retinanet.git'
    self.model_url= ['https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5',
               'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet101_oid_v1.0.0.h5',
               'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet152_oid_v1.0.0.h5']
    
    self.color_code=   {'Cardiomegaly':(124,252,0), 'Aortic enlargement':(135,206,250),
                        'Pleural thickening':(199,21,133),'ILD':(245,245,220), 'Nodule/Mass':(220,20,60),
                        'Pulmonary fibrosis':(0,255,255), 'Lung Opacity':(128,128,0), 'Atelectasis':(255,0,255),
                        'Other lesion':(176,224,230), 'Infiltration':(210,105,30),'Pleural effusion':(105,105,105),
                        'Calcification':(138,43,226) ,'Consolidation':(250,240,230),'Pneumothorax':(100,149,237)}
    
cfg= Cfg()

<h2 style="font-family:verdana;"> <center>📚 Data Processing for Retinanet</center> </h2>




<hr>

In [ ]:
df= pd.read_csv(cfg.train_csv)
print(df.shape)
df= df[df.class_name != 'No finding']
display(df.head())
df.shape

In [ ]:
def load_img(path):
    img= cv2.imread(path)
    img= cv2.resize(img, (cfg.dim, cfg.dim))
    return img

def normalize_cod(df):
    df.x_min= (df.x_min/ df.width)* cfg.dim
    df.x_max= (df.x_max/ df.width)* cfg.dim
    
    df.y_min= (df.y_min/ df.height)* cfg.dim
    df.y_max= (df.y_max/ df.height)* cfg.dim
    return df

df= normalize_cod(df.copy())
df= df.reset_index(drop = True)

df.head()

In [ ]:
classes= df.class_name.unique()
ind= df.class_id.unique()

In [ ]:
##### Color-Code ########
color= cfg.color_code

def show_bb(i):
    df_mini= df[df.image_id==df.image_id[i]]
    path= cfg.img_dir + df.image_id[i] + '.png'
    img= load_img(path)
    rep_class=[]
    font = cv2.FONT_HERSHEY_SIMPLEX 
    for i, row in df_mini.iterrows():
        class_n= row['class_name']
        if class_n in rep_class:
            continue                          # More generalization
        rep_class.append(class_n)
        x_min= int(row['x_min']); x_max= int(row['x_max'])
        y_min= int(row['y_min']); y_max= int(row['y_max'])
        img= cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color[class_n], 2)
        fontScale= (x_max- x_min)*2.5/img.shape[1]
        img= cv2.putText(img, class_n, (x_min, y_min-5), font, fontScale, cv2.LINE_AA)
    return img

In [ ]:
f, ax= plt.subplots(2, 4, figsize=(26, 14))
for i in range(8):
    ax[i//4][i%4].imshow(show_bb(i), aspect='auto')
    ax[i//4][i%4].set_xticks([]); ax[i//4][i%4].set_yticks([])
plt.show()

<h2 style="font-family:verdana;"> <center>Data Spliting</center> </h2>

<hr>

In [ ]:
'''
https://www.kaggle.com/backtracking/smart-data-split-train-eval-for-object-detection
'''

skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=101)
df_folds = df[['image_id']].copy()

df_folds.loc[:, 'bbox_count'] = 1
df_folds = df_folds.groupby('image_id').count()
df_folds.loc[:, 'object_count'] = df.groupby('image_id')['class_id'].nunique()

df_folds.loc[:, 'stratify_group'] = np.char.add(
    df_folds['object_count'].values.astype(str),
    df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
)

df_folds.loc[:, 'fold'] = 0
for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
    df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number

In [ ]:
df_folds.reset_index(inplace=True)

df_valid = pd.merge(df, df_folds[df_folds['fold'] == 0], on='image_id')
df_train = pd.merge(df, df_folds[df_folds['fold'] != 0], on='image_id')

df_train.shape, df_valid.shape

In [ ]:
plt.figure(figsize=(20, 4))
plt.hist(df_train['class_name'], label='Train Data Split')
plt.hist(df_valid['class_name'], label='Test Data Split')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Train set
df_train= df_train[['image_id','x_min', 'y_min', 'x_max', 'y_max', 'class_name']]
df_train.image_id= df_train.image_id.apply(lambda x: '../'+ cfg.img_dir+ x+ '.png')
df_train.iloc[:,1:-1]=df_train.iloc[:,1:-1].astype('int32')

# validation set
df_valid= df_valid[['image_id','x_min', 'y_min', 'x_max', 'y_max', 'class_name']]
df_valid.image_id= df_valid.image_id.apply(lambda x: '../'+ cfg.img_dir+ x+ '.png')
df_valid.iloc[:,1:-1]=df_valid.iloc[:,1:-1].astype('int32')

df_train.head()

In [ ]:
# just Sanity check stuff

#df_train= df_train.drop(df_train[df_train.y_max== df_train.y_min].index[0])
df_valid= df_valid.drop(df_valid[df_valid.y_max== df_valid.y_min].index[0])


<h2 style="font-family:verdana;"> <center>🏛 DRetinanet architecture</center> </h2>

![00194_psisdg11321_113210f_page_2_1](https://user-images.githubusercontent.com/64481847/103896119-7d1bbe80-5117-11eb-893e-a0efbfa3fc31.jpg)


<hr>

<h2 style="font-family:verdana;"> <center>⚙️ RetinaNet installation</center> </h2>


<hr>


In [ ]:
################## RETINANET #######################
!git clone 'https://github.com/fizyr/keras-retinanet.git'

In [ ]:
%%capture

%cd keras-retinanet/
!pip install .

In [ ]:
%%capture

!python setup.py build_ext --inplace

In [ ]:
####### Saving csv and classes files ##########
df_train.to_csv('annotations.csv', index=False, header=None)
df_valid.to_csv('annotations_test.csv', index=False)


print(classes)

with open("classes.csv","w") as file:
    for i, cn in zip(ind, classes):
        file.write('{},{}\n'.format(cn, i))

In [ ]:
import tensorflow
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr ,preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

import requests
import urllib

In [ ]:
pretrained_model = 'snapshots/pretrained_model_101.h5'

model_url = cfg.model_url[1]
urllib.request.urlretrieve(model_url, pretrained_model)

print("Downloaded pretrained model- {} to-'{}'".format(model_url, pretrained_model))

In [ ]:
%ls

In [ ]:
!keras_retinanet/bin/train.py --freeze-backbone \
  --random-transform \
  --workers 0 \
  --weights {pretrained_model} \
  --batch-size 6 \
  --steps 550 \
  --image-min-side 512\
  --image-max-side 512\
  --epochs 50 \
  csv annotations.csv classes.csv

### More Parameters
https://github.com/fizyr/keras-retinanet/blob/master/keras_retinanet/bin/train.py

    oid_parser = subparsers.add_parser('oid')
    oid_parser.add_argument('main_dir', help='Path to dataset directory.')
    oid_parser.add_argument('--version',  help='The current dataset version is v4.', default='v4')
    oid_parser.add_argument('--labels-filter',  help='A list of labels to filter.', type=csv_list, default=None)
    oid_parser.add_argument('--annotation-cache-dir', help='Path to store annotation cache.', default='.')
    oid_parser.add_argument('--parent-label', help='Use the hierarchy children of this label.', default=None)

    csv_parser = subparsers.add_parser('csv')
    csv_parser.add_argument('annotations', help='Path to CSV file containing annotations for training.')
    csv_parser.add_argument('classes', help='Path to a CSV file containing class label mapping.')
    csv_parser.add_argument('--val-annotations', help='Path to CSV file containing annotations for validation (optional).')

    group = parser.add_mutually_exclusive_group()
    group.add_argument('--snapshot',          help='Resume training from a snapshot.')
    group.add_argument('--imagenet-weights',  help='Initialize the model with pretrained imagenet weights. This is the default behaviour.', action='store_const', const=True, default=True)
    group.add_argument('--weights',           help='Initialize the model with weights from a file.')
    group.add_argument('--no-weights',        help='Don\'t initialize the model with any weights.', dest='imagenet_weights', action='store_const', const=False)
    parser.add_argument('--backbone',         help='Backbone model used by retinanet.', default='resnet50', type=str)
    parser.add_argument('--batch-size',       help='Size of the batches.', default=1, type=int)
    parser.add_argument('--gpu',              help='Id of the GPU to use (as reported by nvidia-smi).')
    parser.add_argument('--multi-gpu',        help='Number of GPUs to use for parallel processing.', type=int, default=0)
    parser.add_argument('--multi-gpu-force',  help='Extra flag needed to enable (experimental) multi-gpu support.', action='store_true')
    parser.add_argument('--initial-epoch',    help='Epoch from which to begin the train, useful if resuming from snapshot.', type=int, default=0)
    parser.add_argument('--epochs',           help='Number of epochs to train.', type=int, default=50)
    parser.add_argument('--steps',            help='Number of steps per epoch.', type=int, default=10000)
    parser.add_argument('--lr',               help='Learning rate.', type=float, default=1e-5)
    parser.add_argument('--optimizer-clipnorm', help='Clipnorm parameter for  optimizer.', type=float, default=0.001)
    parser.add_argument('--snapshot-path',    help='Path to store snapshots of models during training (defaults to \'./snapshots\')', default='./snapshots')
    parser.add_argument('--tensorboard-dir',  help='Log directory for Tensorboard output', default='')  # default='./logs') => https://github.com/tensorflow/tensorflow/pull/34870
    parser.add_argument('--tensorboard-freq', help='Update frequency for Tensorboard output. Values \'epoch\', \'batch\' or int', default='epoch')
    parser.add_argument('--no-snapshots',     help='Disable saving snapshots.', dest='snapshots', action='store_false')
    parser.add_argument('--no-evaluation',    help='Disable per epoch evaluation.', dest='evaluation', action='store_false')
    parser.add_argument('--freeze-backbone',  help='Freeze training of backbone layers.', action='store_true')
    parser.add_argument('--random-transform', help='Randomly transform image and annotations.', action='store_true')
    parser.add_argument('--image-min-side',   help='Rescale the image so the smallest side is min_side.', type=int, default=800)
    parser.add_argument('--image-max-side',   help='Rescale the image if the largest side is larger than max_side.', type=int, default=1333)
    parser.add_argument('--no-resize',        help='Don''t rescale the image.', action='store_true')
    parser.add_argument('--config',           help='Path to a configuration parameters .ini file.')
    parser.add_argument('--weighted-average', help='Compute the mAP using the weighted average of precisions among classes.', action='store_true')
    parser.add_argument('--compute-val-loss', help='Compute validation loss during training', dest='compute_val_loss', action='store_true')
    parser.add_argument('--reduce-lr-patience', help='Reduce learning rate after validation loss decreases over reduce_lr_patience epochs', type=int, default=2)
    parser.add_argument('--reduce-lr-factor', help='When learning rate is reduced due to reduce_lr_patience, multiply by reduce_lr_factor', type=float, default=0.1)
    parser.add_argument('--group-method',     help='Determines how images are grouped together', type=str, default='ratio', choices=['none', 'random', 'ratio'])

    # Fit generator arguments
    parser.add_argument('--multiprocessing',  help='Use multiprocessing in fit_generator.', action='store_true')
    parser.add_argument('--workers',          help='Number of generator workers.', type=int, default=1)
    parser.add_argument('--max-queue-size',   help='Queue length for multiprocessing workers in fit_generator.', type=int, default=10)



## 🌄 Thanks for Reading

![](https://i.gifer.com/7ImI.gif)



<div class="alert alert-block alert-info" style="font-size:20px; font-family:verdana;">
 <a target="_blank" style="color:orange;">Do UPVOTE for more Motivation🤞</a>
</div>



<hr><hr><hr>

<hr>

In [ ]:
# Download all Working data 

%cd ..

In [ ]:
import shutil
shutil.make_archive('Keras-Retinanet', 'zip', 'keras-retinanet')

<a href="./Keras-Retinanet.zip">DOWNLOAD ZIP FILE :-)</a>